In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyreadr

import pyreadr
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import math as m
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
import copy as c
from  google.colab import drive
from scipy.stats import spearmanr
from scipy.ndimage.filters import gaussian_filter1d
from scipy.signal import argrelmax
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
import logging

In [ ]:
fs = 60  # training
hours_ = 16  # hours info
bootstrap_rep =2000


data_folder="/content/drive/MyDrive/Kyoto/data"
# data_folder = "F:\\KyotoTemporal\\out\\MLE\\test\\"
# data_folder="F:\\KyotoTemporal\\out\\MLE_TS_1\\01\\"
# out_folder = "F:\\KyotoTemporal\\out\\RNN_out\\RiskEstimate\\"
out_folder = "/content/drive/MyDrive/Kyoto"
## Monthly data store
# base_folder = "F:\\KyotoTemporal\\out\\RNN_out\\Fcast_Actual\\"


day_list = []


#create a logger
# logger = logging.getLogger()

for folder in os.listdir(data_folder):
    df3=pd.DataFrame()
    d = os.path.join('/content/drive/MyDrive/Kyoto/data', folder)
    #     df =pd.read_csv(d)
    result = pyreadr.read_r(d)
    
    print(result.keys())
    
    # input_data = result["user_ts"]  # extract the pandas data frame for object c6 --> MLE data
    input_data = result["input"]  # extract the pandas data frame for object c6

    d1 = input_data.drop(["i"], axis=1)

    # day = pd.to_numeric((d.split('1_')[1]).split('.')[0]) ---> for MLE data
    day = pd.to_numeric(d.split('2015')[1][2:4])
    day_list.append(day)
    out_perm = np.array([])
  # log file
    log_filename = str.format('mylog%d.log' % day)
    logging.basicConfig(filename=log_filename, level=logging.INFO)

    np_filter_d1 = d1.fillna(method='bfill')  # ffill - forward-fill propagate inplace=False

    ##### ---------------  for TESTING ONLY
    # filtered_d1 = c.deepcopy(np_filter_d1.iloc[0:5000,:])

  ##### ---------------  for TESTING ONLY


    ###### ------------ Testing for Entire data
    ###### ------------
    filtered_d1 = c.deepcopy(np_filter_d1)
    ###### ------------
    ###### ------------ Testing for Entire data
    

# 16 hours of data for training
    # train_size=int(fs*hours_*60) --> for MLE 1sec
    train_size=int(fs*hours_)
    col_len = filtered_d1.shape[1]
    
    # seperate training and testing data
    tr_df = c.deepcopy(filtered_d1[0:train_size])
    tst_df = c.deepcopy(filtered_d1[train_size:])
    
# feature varianve ----------------------------------------------------
  # all actual variable
    combine_actual_df =pd.concat([tr_df, tst_df], axis=0)
  # original actal feature variance
    # actual_df_var = combine_actual_df.iloc[:,0:37].var(axis=1)  #-------> Original for MLE
    actual_df_var = combine_actual_df.iloc[:,0:85].var(axis=1)  #-------> modified July for DWT_PE
  # integration of actual feature variance and class such as session, normal, KA, UA
    actual_df = pd.concat([actual_df_var, combine_actual_df.iloc[:,37:col_len]], axis=1)
    
    # scaling to (0,1)
    train_df = c.deepcopy(tr_df)
    scalers={}
    for i in tr_df.columns:
        scaler = MinMaxScaler(feature_range=(0,1))
        s_s = scaler.fit_transform(train_df[i].values.reshape(-1,1))
        s_s=np.reshape(s_s,len(s_s))
        scalers['scaler_'+ i] = scaler
        train_df[i]=s_s
    test_df = c.deepcopy(tst_df)  
    for i in tr_df.columns:
        scaler = scalers['scaler_'+i]
        s_s = scaler.transform(test_df[i].values.reshape(-1,1))
        s_s=np.reshape(s_s,len(s_s))
        scalers['scaler_'+i] = scaler
        test_df[i]=s_s


  # train and test data combine
    data_scaled = pd.concat([train_df, test_df], axis=0)

    
# prepare for writing output -------------------------------
    os.chdir(out_folder) #change the month directory
    # print('Completed current folder --',out_folder)
    month_folder = (d.split('2015')[1][0:2])


    # create a folder for month
    if not os.path.exists(month_folder):
      
      os.makedirs(month_folder)
    # join the month folder to existed directory       
    write_dir = os.path.join(out_folder, month_folder)   
    os.chdir(write_dir) #change the month directory
    print('Completed write --',write_dir)
    
  ## ------------------------------------------------------------------------------------------------
  ##   risk level and threshold
  ## ------------------------------------------------------------------------------------------------
    level, t, t_status = risk_score_generation_colab(filtered_d1,data_scaled, d, day, write_dir,bootstrap_rep)
    
    ## ------------------------------------------------------------------------------------------------
    # logging.info("Done risk_score_generation_colab for entire data ")  
     ## ------------------------------------------------------------------------------------------------
    # print('------- done 0')
    #     # dividing traing and testing with defined train size
#     train_df,test_df = data_scaled[0:train_size], data_scaled[train_size:] 
# class info to be forecasted 
    out_idx = [d1.columns.get_loc("c_ses"), d1.columns.get_loc("Normal"), d1.columns.get_loc("KA"),d1.columns.get_loc("UA")]
    
    # Set the input_sequence_length length - this is the timeframe used to make a single prediction
    input_sequence_length = fs # number of features

    # output_sequence_length = len(out_idx) # number of outputs
    output_sequence_length = 1 # number of outputs

  ## ------------------------------------------------------------------------------------------------
  ##   spliting data to features and class
  ## ------------------------------------------------------------------------------------------------
# 
    x_train, y_train = partition_dataset(input_sequence_length, output_sequence_length, train_df.values,out_idx)
    x_test, y_test = partition_dataset(input_sequence_length, output_sequence_length, test_df.values,out_idx)


  ## ------------------------------------------------------------------------------------------------
  ##   LSTM model generation 
  ## ------------------------------------------------------------------------------------------------
    n_output_neurons = 4
    model = lstm_model_creation(n_output_neurons, x_train)   
#     model = Sequential()
#     # n_output_neurons = output_sequence_length
#     n_output_neurons = 4

#     n_input_neurons = x_train.shape[1] * x_train.shape[2]
# #     n_input_neurons = x_train.shape[2]
#     print(n_input_neurons, x_train.shape[1], x_train.shape[2])
#     model.add(LSTM(n_input_neurons, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2]))) 
#     model.add(Dropout(0.25))

#     model.add(LSTM(int(n_input_neurons/2), return_sequences=False))
#     model.add(Dropout(0.25))

#     model.add(Dense(20, activation='relu'))
#     model.add(Dropout(0.25))

#     # model.add(Dense(output_sequence_length))
#     model.add(Dense(n_output_neurons))

#     model.compile(optimizer='adam', loss='mse',metrics=['accuracy']) 
    # model.summary()
    # Training the model

##### ---------------  for TESTING ONLY
    # epochs = 10
    # batch_size = 2


  ## ------------------------------------------------------------------------------------------------
  ##   TESTING ONLY
  ## ------------------------------------------------------------------------------------------------

    epochs = 50
    batch_size = 10
    early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
    history = model.fit(x_train, y_train,batch_size=batch_size, 
                        epochs=epochs, validation_data=(x_test, y_test))
    ## ------------------------------------------------------------------------------------------------
    ##   write the accuracy for train and validation
    ## ------------------------------------------------------------------------------------------------
    acc_name = file_name_create(day, '_ACCURACY_train_val')
    train_acc=history.history['accuracy']
    train_acc = pd.DataFrame(train_acc)
    val_acc = history.history['val_accuracy']
    val_acc = pd.DataFrame(val_acc)
    all_acc = pd.concat([train_acc,val_acc], axis=1)
    # all_acc = train_acc +  val_acc
    all_acc.columns = ['Train','Validation']                                            

    all_acc.to_csv(write_dir + '/' + acc_name,  index=False, header=True)
  ## ------------------------------------------------------------------------------------------------
    # predict
  # ## ------------------------------------------------------------------------------------------------  
    # 
    pred_e1d1=model.predict(x_test)

    y_pred = scaler.inverse_transform(pred_e1d1)
    y_pred= y_pred.reshape((len(y_test), n_output_neurons))
    #reshape_test= y_test.reshape((len(y_test), n_output_neurons))
    #inv_y_test = scaler.inverse_transform(reshape_test) #---- not needed
    # inverse transform for testing label
    # inv_test= scaler.inverse_transform(y_test.reshape(-1,1)).reshape(y_test.shape) 
    inv_y_test= y_test.reshape((len(y_test), n_output_neurons))

    # # actual risk level from data scale     
    # act_d_r_l = dy_level_risk[train_size:] # for actual risk level
    

  
    # tst_risk_score = risk_score_calculation(test_df,tst_act_scaled_df,d,out_folder)
    # check any numpy array contains Nan value
    if  np.isnan(np.sum(y_pred)) : 
        # y_pred =np.nan_to_num(y_pred) # if true, replace to zero
        col_mean = np.nanmean(y_pred, axis=0)
        
        #Find indices that you need to replace
        inds = np.where(np.isnan(y_pred))

        #Place column means in the indices. Align the arrays using take
        y_pred[inds] = np.take(col_mean, inds[1])
    
    
    ## -------  RMSE accuracy measures  --------------------
    for i in range(0,n_output_neurons):
        #when target attributes are also scaled
        p= mean_squared_error(inv_y_test[:,i], y_pred[:,i], squared=False)
        mse= mean_squared_error(inv_y_test[:,i], y_pred[:,i])
        mae = mean_squared_error(inv_y_test[:,i], y_pred[:,i])
        # when target attributes are not scaled

        out_perm = np.hstack((out_perm, p,mae, mse)) 
#         
        del p,mae,mse
    
    out_perm = np.hstack((day,out_perm))
    
    
    out=pd.DataFrame(out_perm)
    out =out.T
    out_perm=pd.DataFrame(out_perm)
    y_pred=pd.DataFrame(y_pred)
    inv_y_test=pd.DataFrame(inv_y_test)

    df3 = pd.concat([df3,out], axis=0)

  ## ------------------------------------------------------------------------------------------------
    # logging.info("Done RMSE ")  
    ## ------------------------------------------------------------------------------------------------  
    tst_row_dim = (test_df.shape)[0]
    # tst_scale_var=test_df.iloc[fs+1:tst_row_dim,0:37].var(axis=1) --> MLE
    tst_scale_var=test_df.iloc[fs+1:tst_row_dim,0:37].var(axis=1)
    
    pd.DataFrame(tst_scale_var)
    tst_scale_var.reset_index(drop=True,inplace=True)
    
    pred_row_var = pd.concat([tst_scale_var,y_pred], axis=1)
    inv_row_var = pd.concat([tst_scale_var,inv_y_test], axis=1)
    
  
    
# write to output
    day_folder = (d.split('2015')[1][2:4])
    hdr=['fea_var','session','N','A','NA']
    fcast_name = file_name_create(day_folder, '_Fcast_values')
    pred_row_var.to_csv(write_dir + '/' + fcast_name, index=False,header=hdr) 

    acual_name = file_name_create(day_folder, '_Actual_values')
    inv_row_var.to_csv(write_dir + '/' + acual_name, index=False, header=hdr)  

    # target = scale_score['risk']
    # target_ecdf_values, y = ecdf_values(target)
    # change_values, change_idx = peak_change_V2(target_ecdf_values, str(day), save=False)
    # bootstrap_out_idx = bootstrap_out(change_values, change_idx, str(day), btrap_rep)
    
    # tst_level, t = risk_score_generation_colab(filtered_d1,data_scaled, write_dir, d, day, 'Risk',bootstrap_rep)
    tst_act_depnedvarOnly= test_df.iloc[0:(y_pred.shape)[0],37:(test_df.shape)[1]]

    # index rearrange from 0
    tst_act_depnedvarOnly.reset_index(drop=True,inplace=True)
    # test dataset feature variance calculation
    tst_scl_var = test_df.iloc[:,0:37].var(axis=1)
    pd.DataFrame(tst_scl_var)
    tst_scl_var.reset_index(drop=True,inplace=True)
    # # combine between the feature var and fcast values
    tst_scl_var_df=tst_scl_var.iloc[0:(y_pred.shape)[0]]
    tst_scl_var_df=pd.concat([tst_scl_var_df,y_pred],axis=1)
    tst_scl_var_df.columns = ['var', 'c_ses', 'Normal', 'KA', 'UA']
  

  # risk level with fcast values
    test_score = fcast_risk_level_colab_2(tst_act_depnedvarOnly,tst_scl_var_df,t,t_status, day, write_dir)

    ## ------------------------------------------------------------------------------------------------
    # logging.info("Done Risk Level with fcast values ")  
    ## ------------------------------------------------------------------------------------------------  
      

    # confusion matrix of risk levels between test depenent variabls and the forecasted variables
    Risklevel_performance (level, train_size, test_score, y_pred, d, write_dir,month_folder)

     ## ------------------------------------------------------------------------------------------------
    # logging.info("Done MRiskLevel Performance")  
    ## ------------------------------------------------------------------------------------------------  
    
    fcast_name = file_name_create(month_folder, '_RMSE_MAPE_LSTM')
    if (d.split('2015')[1][2:4]) == '01':
        df3.columns = ['Day','S_RMSE','S_MAE','S_MSE','N_RMSE','N_MAE','N_MSE','A_RMSE','A_MAE','A_MSE','UA_RMSE','UA_MAE','UA_MSE']
        # fcast_perm_name = file_name_create(month_folder, '_RMSE_MAPE_LSTM')
        # fcast_name = file_name_create(month_folder, '_RMSE_MAPE_LSTM')
        df3.to_csv(write_dir + '/' + fcast_name,  index=False) 
          
    else:
        # fcast_name = file_name_create(month_folder, '_RMSE_MAPE_LSTM')
        # df3.to_csv(write_dir + '/' + fcast_perm_name,  index=False, mode='a',header=False)
        df3.to_csv(write_dir + '/' + fcast_name,  index=False, mode='a',header=False)

## ------------------------------------------------------------------------------------------------
    # logging.info("Done for day %s ",day)  
    ## ------------------------------------------------------------------------------------------------  
    

In [ ]:
d

In [ ]:

######----------LSTM Model generation  --------------------------
def lstm_model_creation(n_out_nurons,x1_train):

	#LSTM model generation and training
    model_g = Sequential()
    # n_output_neurons = output_sequence_length
    # n_out_nurons = 4

    n_in_nurons = x1_train.shape[1] * x1_train.shape[2]
#     n_input_neurons = x_train.shape[2]
    #print(n_input_neurons, x_train.shape[1], x_train.shape[2])
    model_g.add(LSTM(n_in_nurons, return_sequences=True, input_shape=(x1_train.shape[1], x1_train.shape[2]))) 
    model_g.add(Dropout(0.25))

    model_g.add(LSTM(int(n_in_nurons/2), return_sequences=False))
    model_g.add(Dropout(0.25))

    model_g.add(Dense(20, activation='relu'))
    model_g.add(Dropout(0.25))

    # model.add(Dense(output_sequence_length))
    model_g.add(Dense(n_out_nurons))

    model_g.compile(optimizer='adam', loss='mse',metrics=['accuracy']) 
    # model.summary()
    
    return model_g

###### ---------------------  fcast_risk_level_colab
def fcast_risk_level_colab_2(tst_act_depvals,tst_scaled_var,cut_threshold, cut_status, day, write_dir,):


  copy_tdf = c.deepcopy(tst_act_depvals)
  scale_tscore = c.deepcopy(tst_scaled_var)

  col_list = ['KA', 'UA']
  scale_tscore['AttackSum'] = copy_tdf[col_list].sum(axis=1)

  scale_tscore['ratio_N'] = scale_tscore['Normal'] + scale_tscore['var']
  scale_tscore['ratio_A'] = (scale_tscore['UA'] + scale_tscore['KA']) + scale_tscore['var']

  idx_n = scale_tscore.columns.get_loc("ratio_N")
  idx_a = scale_tscore.columns.get_loc("ratio_A")
  idx_s = copy_tdf.columns.get_loc("c_ses")
  #     def risk_score(data):
  scale_tscore['risk'] = (scale_tscore['ratio_A'] / (scale_tscore['ratio_N'] + scale_tscore['ratio_A'])) * copy_tdf[
      'c_ses']
  if cut_status != 0: ## there are zero changes
    
    if cut_status == 1: ## there are zero changes
      scale_tscore["Level"] = np.where(scale_tscore["risk"] < cut_threshold, 'L', 'M')
    elif cut_status == 2: 
      scale_tscore["Level"] = np.where(scale_tscore["risk"] < cut_threshold, 'M', 'H')
    else:
      scale_tscore["Level"] = np.where(scale_tscore["risk"] < cut_threshold, 'H')
  else: #cut_status == 0 that is the risk level is L/H or L/M/H

    if len(cut_threshold) == 1: # two level
          scale_tscore["Level"] = np.where(scale_tscore["risk"] < cut_threshold[0], 'L', 'H')
    # elif len(cut_off_value[0]) == 0: 
    #   ...
    else: # three level
          min_val = min(cut_threshold)
          max_val = max(cut_threshold)
          scale_tscore.loc[scale_tscore["risk"] < min_val, "Level"] = 'L'
          scale_tscore.loc[(scale_tscore['risk'] >= min_val) & (scale_tscore['risk'] < max_val), "Level"] = 'M'
          scale_tscore.loc[scale_tscore['risk'] >= max_val, "Level"] = 'H'

  _name = file_name_create(day, '_fcast_risk_level')
  scale_tscore.to_csv(write_dir + '/' + _name,  index=False)

  return scale_tscore

######----------Risklevel_performance  --------------------------
def Risklevel_performance (all_level, tr_size, scale_tscore, yhat, day_info, wr_dir,month_info):
	
  daily_info = str(day_info).split('2015')[1][2:4]	
  all_risk_level = c.deepcopy(all_level)
  yhat_df = c.deepcopy(yhat)
  # cm_risk=pd.DataFrame()
  cm=pd.DataFrame()

  df_level = all_risk_level[tr_size:]
  df_level.reset_index(drop=True,inplace=True)
  tst_level=df_level.iloc[0:(yhat_df.shape)[0],:]

  len_tst_risk = len(pd.unique(tst_level['Level']))
  actual_risk = len(pd.unique(all_level['Level']))
  daily_info = str(day_info).split('2015')[1][2:4]

  if actual_risk != len_tst_risk: ## ----- modified July 31 2022
    acc=0
    prec=0
    recall=0
    f1=0
    cm = np.hstack((daily_info,acc,prec,recall,f1))

    cm1=pd.DataFrame(cm)
    cm1=cm1.T
    cm1_name = file_name_create(month_info, '_cm_2_risk_level')

    if str(day_info).split('2015')[1][2:4] =='01':
      cm1.columns=['day','acc','precision','recall','f1']
      cm1.to_csv(write_dir + '/' + cm1_name,  index=False,header=True)
    else:
      cm1.to_csv(write_dir + '/' + cm1_name,  index=False,header=False,mode='a')

  else:   
    if len_tst_risk==2:

      tn, fp, fn, tp = confusion_matrix(tst_level['Level'], scale_tscore['Level']).ravel()
      acc= accuracy_score(tst_level['Level'], scale_tscore['Level']) 
      prec=metrics.precision_score(tst_level['Level'], scale_tscore['Level'], average='weighted')
      recall = metrics.recall_score(tst_level['Level'], scale_tscore['Level'], average='weighted')
      #prec=tp/(tp+fp)
      #recall=tp/(tp+fn) 
      f1 = 2 * (prec * recall) / (prec + recall) 
      #total=(tn+fp+fn+tp)
      #acc = (tp+tn)/total
      #sen =  tp/(tp+fn)
      #spec = tn/(fp+tn)
      daily_info = str(day_info).split('2015')[1][2:4] 
      cm = np.hstack((daily_info,acc,prec,recall,f1))

      cm1=pd.DataFrame(cm)
      cm1=cm1.T
      cm1_name = file_name_create(month_info, '_cm_2_risk_level')

      if str(day_info).split('2015')[1][2:4] =='01':
        cm1.columns=['day','acc','precision','recall','f1']
        cm1.to_csv(write_dir + '/' + cm1_name,  index=False,header=True)
      else:
        cm1.to_csv(write_dir + '/' + cm1_name,  index=False,header=False,mode='a')

    elif  len_tst_risk==1: # one level 
      # tn, fp, fn, tp = confusion_matrix(tst_level['Level'], scale_tscore['Level']).ravel()
      a_val =  len(tst_level['Level'] == test_score['Level'])
      tp=a_val/len(tst_level['Level'])
      acc= accuracy_score(tst_level['Level'], scale_tscore['Level']) 
      prec=metrics.precision_score(tst_level['Level'], scale_tscore['Level'], average='weighted')
      recall = metrics.recall_score(tst_level['Level'], scale_tscore['Level'], average='weighted')
      #prec=tp/(tp+fp)
      #recall=tp/(tp+fn) 
      f1 = 2 * (prec * recall) / (prec + recall) 
      #total=(tn+fp+fn+tp)
      #acc = (tp+tn)/total
      #sen =  tp/(tp+fn)
      #spec = tn/(fp+tn)
      # daily_info = str(day_info).split('2015')[1][2:4]
      cm = np.hstack((daily_info,acc,prec,recall,f1))

      cm1=pd.DataFrame(cm)
      cm1=cm1.T
      cm1_name = file_name_create(month_info, '_cm_2_risk_level')

      if str(day_info).split('2015')[1][2:4] =='01':
        cm1.columns=['day','acc','precision','recall','f1']
        cm1.to_csv(write_dir + '/' + cm1_name,  index=False,header=True)
      else:
        cm1.to_csv(write_dir + '/' + cm1_name,  index=False,header=False,mode='a')
    else:

      MCM= multilabel_confusion_matrix(tst_level['Level'], scale_tscore['Level'])
      # skm.classification_report(tst_level['Level'], scale_tscore['Level'])
      acc = accuracy_score(tst_level['Level'], scale_tscore['Level'])
      prec=metrics.precision_score(tst_level['Level'], scale_tscore['Level'], average='weighted')
      recall = metrics.recall_score(tst_level['Level'], scale_tscore['Level'], average='weighted')
      # recall=prec=tp/(tp+fp)
      # recall=tp/(tp+fn)
      f1 = 2 * (prec * recall) / (prec + recall)
      # sen =  tp/(tp+fn)
      # spec = tn/(fp+tn)
      # daily_info = str(day_info).split('2015')[1][2:4]
      cm = np.hstack((daily_info,acc,prec,recall,f1))

      cm1=pd.DataFrame(cm)
      
      cm1=cm1.T
      cm_name = file_name_create(month_info, '_cm_2_risk_level')

      if str(day_info).split('2015')[1][2:4] =='01':
        cm1.columns=['day','acc','precision','recall','f1']
        cm1.to_csv(wr_dir + '/' + cm_name,  index=False,header=True)
      else:
        cm1.to_csv(wr_dir + '/' + cm_name,  index=False,header=False,mode='a')


######------  split_series  ------------------------------
def split_series(series, n_past, n_future):

  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
    # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)

######------partition_dataset  ------------------------------
def partition_dataset(input_sequence_length, output_sequence_length, data,index_Close):
    x, y = [], []
    data_len = data.shape[0]
    for i in range(input_sequence_length, data_len - output_sequence_length):
        x.append(data[i-input_sequence_length:i,0:index_Close[0]]) #contains input_sequence_length values 0-input_sequence_length * columns
        y.append(data[i:i + output_sequence_length, index_Close]) #contains the prediction values for validation (3rd column = Close),  for single-step prediction
    
    # Convert the x and y to numpy arrays
    x = np.array(x)
    y = np.array(y)
    return x, y


# def mape(actual, pred): 
#     actual, pred = np.array(actual), np.array(pred)
#     return np.mean(np.abs((actual - pred) / actual)) * 100

######---------percentage_error  ---------------------------
def percentage_error(actual, predicted):
    res = np.empty(actual.shape)
    for j in range(actual.shape[0]):
        if actual[j] != 0:
            res[j] = (actual[j] - predicted[j]) / actual[j]
        else:
            res[j] = predicted[j] / np.mean(actual)
    return resp

# def mean_absolute_percentage_error(y_true, y_pred): 
# #     return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100
#     y_true, y_pred = np.array(y_true), np.array(y_pred)
#     return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
######---------mean_absolute_percentage_error  ---------------------------
def mean_absolute_percentage_error(y_true, y_pred): 
#     return np.mean(np.abs(percentage_error(np.asarray(y_true), np.asarray(y_pred)))) * 100
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def fcast_acc_measure(actual, pred):
    fcast_acc=[]
    len_pred=pred_e1d1.shape[1]
    i=0
    while i< len_pred:
        mae = mean_absolute_error(actual[:,i], pred[:,i])
        rmse = m.sqrt(mean_squared_error(actual[:,i], pred[:,i]))
        mape= mean_absolute_percentage_error(actual[:,i], pred[:,i])
        
        p=([mae,rmse,mape])
        fcast_acc=np.concatenate([fcast_acc,p])
        i += 1
    return p


######---------risk_level_2  ---------------------------
# import copy as c
# import numpy as np
# # import file_name_create as fc

# def risk_level_2(in_data, scale_data,w_2_dir, d_info):

#     c_in_data = c.deepcopy(in_data)
#     ci_99 = c_in_data.shape[1] - 1  # 99% CI column
#     ci_99_val = c_in_data.iloc[:,ci_99]
#     diff_idx=np.diff(ci_99_val)
#     r_values=[]
#     # cut_off_value =  np.where(diff_idx != 0)
#     copy_scale_score = c.deepcopy(scale_data)
#     risk_score_idx= copy_scale_score.shape[1]-1 #-----> added July 21/22 for DWT_PE
# 	## check the difference is all the same  or not
#   ## all the same  values either 0, -1,1
#     if (len(ci_99_val.unique()) ==1):

#       if ci_99_val[0] == 0: ## no changes
#         copy_scale_score["Level"] = 'L'
#         # r_values = ((copy_scale_score.iloc[:,risk_score_idx]).mean()).tolist()
#         r_values = [(copy_scale_score.iloc[:,risk_score_idx]).mean()]
#       elif  ci_99_val[0] == 1: ## decreasing
#         copy_scale_score["Level"] = 'M'
#         # r_values = ((copy_scale_score.iloc[:,risk_score_idx]).mean()).tolist()
#         r_values = [(copy_scale_score.iloc[:,risk_score_idx]).mean()]
#       else: ## increasing
#         copy_scale_score["Level"] = 'H'
#         # r_values = ((copy_scale_score.iloc[:,risk_score_idx]).mean()).tolist()
#         r_values = [(copy_scale_score.iloc[:,risk_score_idx]).mean()]
     
#     # elif (len(ci_99_val.unique()) == 2):
#     #     cut_off_value =  np.where(diff_idx != 0)
#     #     for v in range(0, len(cut_off_value[0])):
#     #       r_idx = int(c_in_data.iloc[cut_off_value[0][v]+1,0]) 
#     #       #print(r_idx)
#     #       # get risk score values
#     #       #temp_values = copy_scale_score.iloc[r_idx,copy_scale_score.shape[1]-1]
#     #       temp_values = copy_scale_score.iloc[r_idx,risk_score_idx] #-----> modified July 21/22 for DWT_PE
#     #       r_values.append(temp_values)

#     #       # assign risk level to data
#     #       if len(cut_off_value[0]) == 1: # two level
#     #         copy_scale_score["Level"] = np.where(copy_scale_score["risk"] < r_values[0], 'L', 'H')
#     #       else: # three level
#     #         min_val = min(r_values)
#     #         max_val = max(r_values)
#     #         copy_scale_score.loc[copy_scale_score["risk"] < min_val, "Level"] = 'L'
#     #         copy_scale_score.loc[(copy_scale_score['risk'] >= min_val) & (copy_scale_score['risk'] < max_val), "Level"] = 'M'
#     #         copy_scale_score.loc[copy_scale_score['risk'] >= max_val, "Level"] = 'H'

#     #       l_file_name = file_name_create(d_info, '_risklevel')
#     #       # l_name = str(day_info) + '_risklevel'
#     #       # l_file_name = "%s.csv" % l_name
#     #       copy_scale_score.to_csv(w_2_dir + '/' + l_file_name, index=False)
#     else:  ## different values
     
#         cut_off_value =  np.where(diff_idx != 0)
#         for v in range(0, len(cut_off_value[0])):
#           r_idx = int(c_in_data.iloc[cut_off_value[0][v]+1,0]) 
#           #print(r_idx)
#           # get risk score values
#           #temp_values = copy_scale_score.iloc[r_idx,copy_scale_score.shape[1]-1]
#           temp_values = copy_scale_score.iloc[r_idx,risk_score_idx] #-----> modified July 21/22 for DWT_PE
#           r_values.append(temp_values)

#           # assign risk level to data
#           if len(cut_off_value[0]) == 1: # two level
#             copy_scale_score["Level"] = np.where(copy_scale_score["risk"] < r_values[0], 'L', 'H')
#           else: # three level
#             min_val = min(r_values)
#             max_val = max(r_values)
#             copy_scale_score.loc[copy_scale_score["risk"] < min_val, "Level"] = 'L'
#             copy_scale_score.loc[(copy_scale_score['risk'] >= min_val) & (copy_scale_score['risk'] < max_val), "Level"] = 'M'
#             copy_scale_score.loc[copy_scale_score['risk'] >= max_val, "Level"] = 'H'

#           l_file_name = file_name_create(d_info, '_risklevel')
#           # l_name = str(day_info) + '_risklevel'
#           # l_file_name = "%s.csv" % l_name
#           copy_scale_score.to_csv(w_2_dir + '/' + l_file_name, index=False)

#     return copy_scale_score,r_values
######   ---------------risk_level_3 
import copy as c
import numpy as np

def risk_level_3(in_data, scale_data,w_2_dir, d_info):

    c_in_data = c.deepcopy(in_data)
    ci_99 = c_in_data.shape[1] - 1  # 99% CI column
    ci_99_val = c_in_data.iloc[:,ci_99]
    diff_idx=np.diff(ci_99_val)
    r_values=[]
    r_status=[] ## identification of the ci_99_val whether 0,1,2,3
    copy_scale_score = c.deepcopy(scale_data)
    risk_score_idx= copy_scale_score.shape[1]-1 #-----> added July 21/22 for DWT_PE
	## check the difference is all the same  or not
    if (len(ci_99_val.unique()) ==1):
		  
        if ci_99_val[0] == 0: ## no changes
            r_values = ((copy_scale_score.iloc[:,risk_score_idx]).mean())
            copy_scale_score["Level"] = np.where(copy_scale_score["risk"] < r_values, 'L', 'M')
            r_status=1
                #r_values = ((copy_scale_score.iloc[:,risk_score_idx]).mean()).tolist()
                #r_values = [(copy_scale_score.iloc[:,risk_score_idx]).mean()]
        elif  ci_99_val[0] == 1: ## decreasing
            r_values = ((copy_scale_score.iloc[:,risk_score_idx]).mean())
            copy_scale_score["Level"] = np.where(copy_scale_score["risk"] < r_values[0], 'M', 'H')
            r_status=2
                # r_values = ((copy_scale_score.iloc[:,risk_score_idx]).mean()).tolist()
            #r_values = [(copy_scale_score.iloc[:,risk_score_idx]).mean()]
        else: ## increasing
            copy_scale_score["Level"] = 'H'
                # r_values = ((copy_scale_score.iloc[:,risk_score_idx]).mean()).tolist()
            r_values = [(copy_scale_score.iloc[:,risk_score_idx]).mean()]
            r_status=3
    else:  ## different values

        r_status=0
        cut_off_value =  np.where(diff_idx != 0)
        for v in range(0, len(cut_off_value[0])):
          r_idx = int(c_in_data.iloc[cut_off_value[0][v]+1,0]) ## due to the diff calcualtion, the original index should be added by 1
          #print(r_idx)
          # get risk score values
          #temp_values = copy_scale_score.iloc[r_idx,copy_scale_score.shape[1]-1]
          temp_values = copy_scale_score.iloc[r_idx,risk_score_idx] #-----> modified July 21/22 for DWT_PE
          r_values.append(temp_values)

          # assign risk level to data
          if len(cut_off_value[0]) == 1: # two level
            copy_scale_score["Level"] = np.where(copy_scale_score["risk"] < r_values[0], 'L', 'H')
          else: # three level
            min_val = min(r_values)
            max_val = max(r_values)
            copy_scale_score.loc[copy_scale_score["risk"] < min_val, "Level"] = 'L'
            copy_scale_score.loc[(copy_scale_score['risk'] >= min_val) & (copy_scale_score['risk'] < max_val), "Level"] = 'M'
            copy_scale_score.loc[copy_scale_score['risk'] >= max_val, "Level"] = 'H'

          l_file_name = file_name_create(d_info, '_risklevel')
          # l_name = str(day_info) + '_risklevel'
          # l_file_name = "%s.csv" % l_name
          copy_scale_score.to_csv(w_2_dir + '/' + l_file_name, index=False)

    return copy_scale_score,r_values,r_status  

######--------spearman_cor_test  ----------------------------
def spearman_cor_test(data):
    # calculate spearman's correlation
    correlation = pd.DataFrame()
    coef, p = spearmanr(data['risk'], data['KA'])
    coef1, p1 = spearmanr(data['risk'], data['Normal'])
    coef2, p2 = spearmanr(data['risk'], data['c_ses'])
    coef3, p3 = spearmanr(data['risk'], data['var'])
    r = [coef, coef1, coef2, coef3, p, p1, p2, p3]
    correlation = correlation.append(r, ignore_index=True)
    correlation = correlation.transpose()
    return correlation

######----------file_name_create  --------------------------
def file_name_create(f_n, f_info):
    f_name = str(f_n) + f_info
    created_file_name = "%s.csv" % f_name
    return created_file_name


######-------------risk_score_generation_colab  -----------------------
def risk_score_generation_colab(fileter_df,filtered_scaled_data, current_data, day_info, w_2_dir,btrap_rep):
    spearman_cor = pd.DataFrame()
    df_scaled = c.deepcopy(filtered_scaled_data)
    # feature variance score
    col_len = filtered_scaled_data.shape[1]
    #----- data is MLE the, the column size is 41 --------------------------------
    # _scale_var = df_scaled.iloc[:, 0:37].var(axis=1)
    # actual_scaled_df = pd.concat([_scale_var, df_scaled.iloc[:, 37:col_len]], axis=1)

     #----- data is DWT_PE the, the column size is 89 --------------------------------
    _scale_var = df_scaled.iloc[:, 0:85].var(axis=1)
    actual_scaled_df = pd.concat([_scale_var, df_scaled.iloc[:, 85:col_len]], axis=1) 
    actual_scaled_df.columns = ['var', 'c_ses', 'Normal', 'KA', 'UA']

    copy_df = c.deepcopy(fileter_df)
    scale_score = c.deepcopy(actual_scaled_df)
    col_list = ['KA', 'UA']
    scale_score['AttackSum'] = copy_df[col_list].sum(axis=1)


    scale_score['ratio_N'] = scale_score['Normal'] + scale_score['var']
    scale_score['ratio_A'] = (scale_score['UA'] + scale_score['KA']) + scale_score['var']

    idx_n = scale_score.columns.get_loc("ratio_N")
    idx_a = scale_score.columns.get_loc("ratio_A")
    idx_s = fileter_df.columns.get_loc("c_ses")
    #     def risk_score(data):
    scale_score['risk'] = (scale_score['ratio_A'] / (scale_score['ratio_N'] + scale_score['ratio_A'])) * fileter_df[
        'c_ses']


    print('Completed risk score--', str(day_info))

    # spearman correlation between risk score and other varuables
    s = spearman_cor_test(scale_score)

    spearman_cor = spearman_cor.append(s, ignore_index=True)
    #     # create a folder for month

    print('Completed spearman --', str(day_info))

    ### write to outfile

    # os.chdir(current_folder) #change the month directory
    # print('Completed current folder --',current_folder)
    m_folder = (current_data.split('2015')[1][0:2])


    # # create a folder for month
    # if not os.path.exists(month_folder):
      
    #   os.makedirs(month_folder)
    # # join the month folder to existed directory       
    # write_dir = os.path.join(current_folder, month_folder)   
    # os.chdir(write_dir) #change the month directory
    # print('Completed write --',write_dir)

    
       
    file_name = file_name_create(day_info, '_riskscore')
    cor_file_name = file_name_create(m_folder, 'Spearman_2_rs')


    print('Writing an output --', str(day_info))

    
    scale_score.to_csv(file_name, index=False)

    # if str(day_info).split('2015')[1][3:4] =='1':  ----------------->MLE
    if str(current_data).split('2015')[1][2:4] =='01':
      spearman_cor.columns = ['KA', 'N', 'Ses', 'var', 'p_KA', 'p_N', 'p_Ses', 'p_var']
      spearman_cor['day']=pd.Series(day_info)
      spearman_cor.to_csv(w_2_dir + '/' + cor_file_name, header=True, index=False)
    else:
      spearman_cor['day']=pd.Series(day_info)
      spearman_cor.to_csv(w_2_dir + '/' + cor_file_name, index=False, header=False, mode='a')

   # spearman_cor.to_csv(w_2_dir + '/' + cor_file_name, index=True, mode='a')
    # scale_score.to_csv(write_dir + '/' + file_name, index=False)
    # spearman_cor.to_csv(write_dir + '/' + cor_file_name, index=False, mode='a')

    print('Calcualting risk score for --', str(day_info))
    target = scale_score['risk']
    target_ecdf_values, y = ecdf_values(target)

    print('Identifying peak change for --', str(day_info))
    change_values, change_idx = peak_change_V2(target_ecdf_values, str(day_info), save=False)
    #     idx_CI = bootstrap(change_idx_values, bootstrap_rep, func=np.mean)
    print('done for peak --', str(day_info))
    bootstrap_out_idx = bootstrap_out(change_values, change_idx, str(day_info), btrap_rep)
    print('done for bootstrap --', str(day_info))

    level_out, risk_cut_off,r_status = risk_level_3(bootstrap_out_idx, scale_score, w_2_dir, day_info)

    return level_out,risk_cut_off,r_status

   

######-------------ecdf_values  -----------------------
def ecdf_values(x):
    """
    Generate values for empirical cumulative distribution function
    
    Params
    --------
        x (array or list of numeric values): distribution for ECDF
    
    Returns
    --------
        x (array): x values
        y (array): percentile values
    """
    
    # Sort values and find length
    x = np.sort(x)
    n = len(x)
    # Create percentiles
    y = np.arange(1, n + 1, 1) / n
    return x, y

######-------------peak_change_V2  -----------------------
def peak_change_V2(x, pig_name, save=False ):
    # fig = plt.figure(figsize=(10,6))
    f = plt.figure()
    # gs = gridspec.GridSpec(5, 1)

    # k = np.gradient(a1_target)
    k = np.gradient(x)
    # ax0 = plt.subplot(gs[0])
    # ax0.plot(k)


    k2 = np.gradient(k)
    # ax1 = plt.subplot(gs[1])
    # ax1.plot(k2)
    # plt.savefig('2st derivative' + '.pdf')
    # plt.plot(k2)
    # f.savefig('2st derivativer' + '.pdf')
    # plt.close()

    k3= np.clip(np.abs(np.gradient(k2)), 0.0001, 2)
    # ax3 = plt.subplot(gs[3])
    # ax3.plot(k3)
    # plt.plot(k3)
    # # plt.savefig('absolute value ' + '.pdf')
    # f.savefig('absolute valuer ' + '.pdf')
    # plt.close()

    smoothed_k = gaussian_filter1d(k3, 20)
    # ax4 = plt.subplot(gs[4])
    # ax4.plot(smoothed_k)
    # plt.savefig('Smoothing applied' + '.pdf')
    # plt.plot(smoothed_k)
    # f.savefig('Smoothing appliedr' + '.pdf')
    # plt.close()

    max_idx = argrelmax(smoothed_k)[0]
    # print(max_idx)


### when plot

    # if save:
    #     plt.plot(k)
    #     plt_name = pig_name
    #     f.savefig(pig_name + '.pdf')

    #     plt.plot(k2)
    #     f.savefig(pig_name + '2stDeri.pdf')

    #     plt.plot(k3)
    #     # plt.savefig('absolute value ' + '.pdf')
    #     f.savefig(pig_name + 'absolute.pdf')

    #     plt.plot(smoothed_k)
    #     f.savefig(pig_name +'Smoothing.pdf')

    #     fig, ax = plt.subplots()
    #     ax.set_title('Risk score evaluation')
    #     ax.set_xlabel('time')
    #     ax.set_ylabel('Risk score evaluation')
    #     ax.plot(x)
    #     ax.scatter(max_idx, x[max_idx], marker='o', color='red')
    #     plt.show()
    #     # plt.savefig('slope change mark' + '.png')
    #     f.savefig(pig_name +'slope_change_mark.pdf')

    #     plt.close()

    return x[max_idx], max_idx

######-------------bootstrap  -----------------------
def bootstrap(b_data, n_rep, func=np.mean):

    """
    Generate `n` bootstrap samples, evaluating `func`
    at each resampling. `bootstrap` returns a function,
    which can be called to obtain confidence intervals
    of interest.
    """
    simulations = list()
    sample_size = len(b_data)
    #     xbar_init = np.mean(data)
    xbar_init = np.var(b_data)
    for c in range(n_rep):
        itersample = np.random.choice(b_data, size=sample_size, replace=True)
        simulations.append(func(itersample))
    simulations.sort()

    def ci(p):
        """
        Return 2-sided symmetric confidence interval specified
        by p.
        """
        u_pval = (1 + p) / 2.
        l_pval = (1 - u_pval)
        l_indx = int(np.floor(n_rep * l_pval))
        u_indx = int(np.floor(n_rep * u_pval))
        return (simulations[l_indx], simulations[u_indx])

    return (ci)

######-------------bootstrap_out  -----------------------
def bootstrap_out(c_data,c_idx,f_name, rep):
    total_value = []
    total_value2 = []
    total_value3 = []

    # copy_data = c.deepcopy(x[max_idx])
    sample = c_data[0]

    for i in range(0, c_data.shape[0]):
        sample = np.append(sample, c_data[i])
		
        var_sample = sample.var()
        #     var_sample = s.mean(sample)
        boot = bootstrap(sample, rep)
		# len(cintervals)= 3
        cintervals = [boot(j) for j in (.90, .95, .99)]

        #         cintervals = [boot(i) for i in range(len(ci_range)-1) ]
        k = 0
        while k <= len(cintervals) - 1:
            # check whther a var is in the CI
            if (var_sample >= cintervals[k][0]) & (var_sample <= cintervals[k][1]):
                out = 1  ### print('Y')
            else:
                out = 0

            if k == 0:
                out_value = [i, var_sample, cintervals[k][0], cintervals[k][1], out]
                total_value.append(out_value)
            elif k == 1:
                out_value2 = [cintervals[k][0], cintervals[k][1], out]
                total_value2.append(out_value2)
            else:
                out_value3 = [cintervals[k][0], cintervals[k][1], out]
                total_value3.append(out_value3)

            k = k + 1
    #     out_value = [i,cintervals[0][0],cintervals[0][1],cintervals[1][0],cintervals[1][1],cintervals[2][0],cintervals[2][1]]

    #     out_value = [i,var_sample,ci_sample[0],ci_sample[1],out]
    total_value = pd.DataFrame(total_value)
    total_value2 = pd.DataFrame(total_value2)
    total_value3 = pd.DataFrame(total_value3)
    cg_idx = pd.DataFrame(c_idx[0:]) # index for change point --> modified july 26 from 1 to 0

    all_CIs = pd.concat([cg_idx, total_value, total_value2, total_value3], axis=1)
    all_CIs.columns  = ['id', 'idx', 'var','.9Low', '.9up', 'out_9', '.95L', '.95U', 'out_95', '.99L', '.99U', 'out_99']
    out_name= f_name + '_' + str(rep) + '_bootstrapC'
    out_file_name = "%s.csv" % out_name
    # all_CIs=pd.DataFrame(all_CIs,columns=['idx','.9Low','.9up','out_9','.95L','.95U','out_95','.99L','.99U','out_99',])
    all_CIs.to_csv(out_file_name, index=False)
    # all_CIs.to_csv('bootstrapVar5000_CI_ratio.csv', index=False)

    return all_CIs